# Script part

In [ ]:
import json
from pathlib import Path
import shutil
import re
import subprocess
import sys
import stat

In [ ]:
spec_home = "/home/hchaudha/spec"

In [ ]:
def check_regex_in_file(file_path, regex_pattern):
    # Read the content of the file
    with open(file_path, 'r') as file:
        content = file.read()

    # Try to compile the regex pattern
    try:
        pattern = re.compile(regex_pattern)
    except re.error:
        print("Invalid regex pattern.")
        return False

    # Search for matches
    if pattern.search(content):
        return True
    else:
        return False

def replace_current_file(file_path, original_str, replaced_str,**kwargs):
    matched_strings = []

    def callback(match):
        matched_strings.append(match.group(0))  # Store the matched string
        return replaced_str  # Replace with the new string

    with open(file_path, 'r') as file:
        data = file.read()

    # Use re.sub with callback to replace and collect matched strings
    data, replaced_status = re.subn(original_str, callback, data, **kwargs)   

    if replaced_status != 0:
        print(f"""
Replaced in File: {file_path}
Original String: {original_str}
Replaced String: {replaced_str}
Matched Strings: {matched_strings}
""")
    else:
        raise Exception(f"""
!!!!FAILED TO REPLACE!!!!
File path: {file_path}
Original String: {original_str}
Replaced String: {replaced_str}
""")

    with open(file_path, 'w') as file:
        file.write(data)

def is_continuous(lst:list):
    # my_list = ["AZ", "BA", "BB", "BC"]
    # print(is_continuous(my_list))  # Should return True

    # my_list = ["AA", "AB", "AC", "AD"]
    # print(is_continuous(my_list))  # Should return True

    # my_list = ["AA", "AB", "AD", "AC"]
    # print(is_continuous(my_list))  # Should return False

    for i in range(len(lst) - 1):
        current = lst[i]
        next_elem = lst[i + 1]

        # Determine the next expected element
        if current[1] == 'Z':
            # If the second character is Z, the next expected should start the next sequence
            expected_next_first = chr(ord(current[0]) + 1)
            expected_next_second = 'A'
        else:
            # Otherwise, continue by incrementing the second character
            expected_next_first = current[0]
            expected_next_second = chr(ord(current[1]) + 1)

        expected_next_elem = expected_next_first + expected_next_second

        # If the next element is not the expected one, return False
        if next_elem != expected_next_elem:
            return False

    return True  # All elements are continuous

def verify_data_dict(data_dict:dict):
  for new_folder_name in data_dict.keys():
    data = data_dict[new_folder_name]
    if not data['new_run_parent'].exists():
      raise Exception(f"""{data['new_run_parent']=} does not exist!""")
    if not data['old_Ev_path'].exists():
      raise Exception(f"""{data['old_Ev_path']=} does not exist!""")
    
    if data["Ev_is_present"]:
      if data["copy_ID"]:
        raise Exception("When Ev_is_present is set to true copy_ID and copy_bin should be false.")
      if data["copy_bin"]:
        raise Exception("When Ev_is_present is set to true copy_ID and copy_bin should be false.")

      Ev_parent_folder_path = data['new_run_parent']/new_folder_name
      if not Ev_parent_folder_path.exists():
        raise Exception(f"Ev_is_present is set to true but {Ev_parent_folder_path=} does not exist.")

      Ev_folder_path = Ev_parent_folder_path/"Ev"
      if not Ev_folder_path.exists():
        raise Exception(f"Ev_is_present is set to true but {Ev_folder_path=} does not exist.")

    if data["copy_ID"]:
      old_ID_path = data['old_Ev_path'].parent/"ID"
      if not old_ID_path.exists():
        raise Exception(f"ID folder marked for copy but {old_ID_path=} does not exist.")    
    
    if data["copy_bin"]:
      old_bin_path = data['old_Ev_path']/"bin"
      if not old_bin_path.exists():
        raise Exception(f"bin folder marked for copy but {old_bin_path=} does not exist.")    
      
    levs_to_copy = data["levs_to_copy"]
    for lev_dict in levs_to_copy:
      segments_to_copy = lev_dict["segments_to_copy"]

      # Check that the segments are continuous and sorted
      if not is_continuous(segments_to_copy):
        raise Exception(f"{segments_to_copy=} is not a continuous list!")

      if lev_dict["this_lev_is_continuation"]:
        # Make sure that the new lev name and the old lev name are the same
        if lev_dict["old_Lev_name"] != lev_dict["new_Lev_name"]:
          raise Exception(f'{lev_dict["this_lev_is_continuation"]=} is set to true but the old lev name {lev_dict["old_Lev_name"]=} is different from the new one {lev_dict["new_Lev_name"]=}.')

        if ("new_Lev_for_AMR_tolerance" in lev_dict) or ("new_Lev_for_GrDomain_and_AmrDriver" in lev_dict):
          raise Exception(f'{lev_dict["this_lev_is_continuation"]=} is set to true but keys new_Lev_for_AMR_tolerance or new_Lev_for_GrDomain_and_AmrDriver are defined!')

        if ("AH_factor" in lev_dict):
          raise Exception(f'{lev_dict["this_lev_is_continuation"]=} is set to true but key AH_factor is defined!')

      else:
        # Check the values of new new_Lev_for_AMR_tolerance and new_Lev_for_GrDomain_and_AmrDriver
        new_Lev_for_AMR_tolerance = lev_dict["new_Lev_for_AMR_tolerance"]
        new_Lev_for_GrDomain_and_AmrDriver = lev_dict["new_Lev_for_GrDomain_and_AmrDriver"]
        # TODO deal with 45 55 etc as 4.5  5.5
        if (isinstance(new_Lev_for_AMR_tolerance,int)) or (isinstance(new_Lev_for_AMR_tolerance,float)):
          pass
        else:
          raise Exception(f"{new_Lev_for_AMR_tolerance=} should be a float or an int.")

        if not isinstance(new_Lev_for_GrDomain_and_AmrDriver,int):
          raise Exception(f"{new_Lev_for_GrDomain_and_AmrDriver=} should be an int.")
        # If this_lev_is_continuation is False and levs have same name then print a warning
        if lev_dict["old_Lev_name"] == lev_dict["new_Lev_name"]:
          if data["new_run_parent"]/new_folder_name == data["old_Ev_path"].parent:
            print(f'\nWARNING!!\n {lev_dict["this_lev_is_continuation"]=} is set to false but the new lev name {lev_dict["old_Lev_name"]=} is same as the old one {lev_dict["new_Lev_name"]=}.')


      for folder in segments_to_copy:
        # Check that the levs we want to copy exist
        old_folder_path = data['old_Ev_path']/(lev_dict["old_Lev_name"]+"_"+folder)
        if not old_folder_path.exists():
          raise Exception(f"{old_folder_path=} does not exist")

        # Check that the files being replace exist
        if folder == segments_to_copy[-1]:
          files_to_change_in_the_new_lev = lev_dict["files_to_change_in_the_new_lev"]
          for file_name in files_to_change_in_the_new_lev:
            file_path = old_folder_path/file_name
            if not file_path.exists():
              raise Exception(f"{file_path=} is supposed to be change in the new lev but it does not exist in the old folders lev.")

            # Check that the lists original_str and replaced_str have the same length
            original_str = files_to_change_in_the_new_lev[file_name]["original_str"]
            replaced_str = files_to_change_in_the_new_lev[file_name]["replaced_str"]
            if len(original_str) != len(replaced_str):
              raise Exception(f"{original_str=} and {replaced_str=} have different lengths!")
            for string_to_be_replaced in original_str:
              if not check_regex_in_file(file_path,string_to_be_replaced):
                raise Exception(f"{string_to_be_replaced=} not found in the file {file_path=}!")

def MakeNextSegment(EV_folder_path:Path, previous_segment_path:Path):
  command = f"cd {EV_folder_path} && {spec_home}/Support/bin/MakeNextSegment -d {previous_segment_path} -t . -S"
  status = subprocess.run(command, capture_output=True, shell=True, text=True)
  if status.returncode == 0:
    print(f"Succesfully ran MakeNextSegment in {EV_folder_path}: \n {status.stdout}")
  else:
    sys.exit(
        f"MakeNextSegment failed in {EV_folder_path} with error: \n {status.stdout} \n {status.stderr}")
    
def get_next_segment(lst):
    # my_list = ["AA", "AB", "AC", "AD", "BA"]
    # print(get_next_segment(my_list))  # Should return "BB"

    # my_list = ["AZ"]
    # print(get_next_segment(my_list))  # Should return "BA"

    # my_list = []
    # print(get_next_segment(my_list))  # Should return "AA"

    if not lst:
        return "AA"  # Return "AA" if the list is empty

    # Sort the list to ensure that elements are in the correct order
    lst.sort()

    # Retrieve the last element from the sorted list
    last_elem = lst[-1]

    # Determine the next element
    if last_elem[1] == 'Z':
        # If the second character is 'Z', increment the first character and set the second to 'A'
        next_first = chr(ord(last_elem[0]) + 1)
        next_second = 'A'
    else:
        # Otherwise, simply increment the second character
        next_first = last_elem[0]
        next_second = chr(ord(last_elem[1]) + 1)

    next_elem = next_first + next_second
    return next_elem

In [ ]:
def link_or_copy(source_path: Path, destination_path: Path, link_or_copy_folders: str):
    if link_or_copy_folders == "link":
        # Create a symbolic link
        if not destination_path.exists():
            destination_path.symlink_to(source_path)
            print(f"Created a symbolic link from {source_path} to {destination_path}")
        else:
            print(f"Destination {destination_path} already exists. Cannot create a link.")
    
    elif link_or_copy_folders == "copy":
        # Copy the directory using shutil when using pathlib
        if not destination_path.exists():
            shutil.copytree(source_path, destination_path)
            print(f"Copied {source_path} to {destination_path}")
            # When copying add write permission as well
            add_write_permission(destination_path)
        else:
            print(f"Destination {destination_path} already exists. Cannot copy.")
    
    else:
        print("Invalid option. Please use 'link' or 'copy'.")

def add_write_permission(target_dir:Path):
  if not target_dir.exists():
    raise Exception(f"Trying to change the permissions of the folder {target_dir} which does not exist.")
  if target_dir.is_symlink():
    raise Exception(f"Trying to get write permission on a symlink folder {target_dir}. This should not be happening.")
  else:
    command = f"chmod u+w -R {target_dir.resolve(strict=True)}"
    status = subprocess.run(command, capture_output=True, shell=True, text=True)
    if status.returncode == 0:
      print(f"Succesfully added execute permission to {target_dir}")
    else:
      sys.exit(
          f"{command}\nfailed with error: \n {status.stderr}")

def copy_all_files(src:Path, dst:Path, exclude_files:list[str]):
  if not src.exists():
    raise Exception(f"{src=} does not exist")
  if not dst.exists():
    raise Exception(f"{dst=} does not exist")

  # Iterate through the source directory
  for path in src.iterdir():
    if path.is_dir():
      # If it's a directory, do not copy
      continue
    else:
      excluded_file = False
      for excluded in exclude_files:
        if excluded in str(path):
          excluded_file = True
          continue
      # If it's not an excluded file, copy it
      if not excluded_file:
        shutil.copy2(path, dst)

def change_AmrTolerances(segement_path:Path, new_lev:int, AH_factor:float=1):
  file_path = segement_path/"AmrTolerances.input"

  TruncationErrorMax = 0.000216536 * 4**(-new_lev)
  replace_current_file(file_path,r"TruncationErrorMax=.*;",f'TruncationErrorMax={TruncationErrorMax};')

  ProjectedConstraintsMax = 0.216536 * 4**(-new_lev)
  replace_current_file(file_path,r"ProjectedConstraintsMax=.*;",f'ProjectedConstraintsMax={ProjectedConstraintsMax};')
  TruncationErrorMaxA = TruncationErrorMax*1.e-4
  replace_current_file(file_path,r"TruncationErrorMaxA=.*;",f'TruncationErrorMaxA={TruncationErrorMaxA};')
  TruncationErrorMaxB = TruncationErrorMax*1.e-4
  replace_current_file(file_path,r"TruncationErrorMaxB=.*;",f'TruncationErrorMaxB={TruncationErrorMaxB};')

  AhMaxRes = TruncationErrorMax / AH_factor
  replace_current_file(file_path,r"AhMaxRes=.*;",f'AhMaxRes={AhMaxRes};')
  AhMinRes = AhMaxRes / 10.0
  replace_current_file(file_path,r"AhMinRes=.*;",f'AhMinRes={AhMinRes};')

  AhMaxTrunc = TruncationErrorMax / AH_factor
  replace_current_file(file_path,r"AhMaxTrunc=.*;",f'AhMaxTrunc={AhMaxTrunc};')
  AhMinTrunc = AhMaxTrunc / 100.0
  replace_current_file(file_path,r"AhMinTrunc=.*;",f'AhMinTrunc={AhMinTrunc};')

def change_GrDomain_and_AmrDriver_Level(segement_path:Path, new_lev:int):
  AmrDriver_path = segement_path/"AmrDriver.input"
  replace_current_file(AmrDriver_path,r"Level\s? =\s? \d;",f"Level = {new_lev};")

  GrDomain_path = segement_path/"GrDomain.input"
  # There is also a WarningLevel = 0; That should not be changed
  replace_current_file(GrDomain_path,r"Level\s?=\s? \d;",f"Level = {new_lev};")

def create_new_folder(folder_name:str,data_dict:dict):
  folder_dict = data_dict[folder_name]
  new_run_parent:Path = folder_dict["new_run_parent"]
  new_run_path = new_run_parent/folder_name
  old_Ev_path:Path = folder_dict["old_Ev_path"]
  link_or_copy_folders = folder_dict["link_or_copy_folders"]
  copy_ID = folder_dict["copy_ID"]
  old_ID_path = old_Ev_path/"ID"
  copy_bin = folder_dict["copy_bin"]
  old_bin_path = old_Ev_path/"bin"

  levs_to_copy = folder_dict["levs_to_copy"]
  
  new_run_path.mkdir(exist_ok=True)
  new_Ev_path = new_run_path/"Ev"
  new_ID_path = new_run_path/"ID"
  new_bin_path = new_run_path/"Ev/bin"
  new_Ev_path.mkdir(exist_ok=True)
  if copy_ID:
    link_or_copy(old_ID_path,new_ID_path,link_or_copy_folders)
  if copy_bin:
    link_or_copy(old_bin_path,new_bin_path,link_or_copy_folders)

  levs_to_copy = folder_dict["levs_to_copy"]
  for lev_dict in levs_to_copy:
    segments_to_copy = lev_dict["segments_to_copy"]

    for segment in segments_to_copy:
      old_segment_path = folder_dict['old_Ev_path']/(lev_dict["old_Lev_name"]+"_"+segment)
      new_segment_path = new_Ev_path/(lev_dict["new_Lev_name"]+"_"+segment)
      link_or_copy(old_segment_path,new_segment_path,link_or_copy_folders)

    # Get the last segment copied or linked
    last_segment_path = new_Ev_path/(lev_dict["new_Lev_name"]+"_"+segments_to_copy[-1])
    last_segment_bin_path = new_Ev_path/(lev_dict["new_Lev_name"]+"_"+segments_to_copy[-1])/"bin"

    # Make the next segment
    new_segment_path = new_Ev_path/(lev_dict["new_Lev_name"]+"_"+get_next_segment(segments_to_copy))
    new_segment_path.mkdir()
    # Copy the bin folder
    link_or_copy(last_segment_bin_path,new_segment_path/"bin","copy")
    copy_all_files(last_segment_path, new_segment_path, ["SpEC."])

    # Add user write permission
    add_write_permission(new_segment_path)

    # Change the Restart option in the Evolution.input
    last_segment_checkpoint_folder_path = last_segment_path/"Run/Checkpoints/"
    if not last_segment_checkpoint_folder_path.exists():
      raise Exception(f"{last_segment_checkpoint_folder_path=} does not exist!")

    if lev_dict["this_lev_is_continuation"]:
      # For continuation runs just change the FromLastStep
      check_regex_in_file(new_segment_path/"Evolution.input",r"Restart\s*=\s*FromLastStep\(.*\);")
      replace_current_file(new_segment_path/"Evolution.input",r"Restart\s*=\s*FromLastStep\(.*\);",f"Restart    = FromLastStep(FilenamePrefix={last_segment_checkpoint_folder_path}/;);")
    else:
      check_regex_in_file(new_segment_path/"Evolution.input",r"Restart\s*=\s*FromLastStep\(.*\);")
      replace_current_file(new_segment_path/"Evolution.input",r"Restart\s*=\s*FromLastStep\(.*\);",f"Restart    = FromLastStep(FilenamePrefix={last_segment_checkpoint_folder_path}/;GrGlobalVarsCheckpoint = Interpolated(FlagAllSubdomainsAsChanged = yes; ResetFilterFunctionsForAMR = yes; StartAmrWithMinimumExtents = yes; ResolutionChanger=Spectral; Interpolator =ParallelAdaptive(TopologicalInterpolator=CardinalInterpolator;); DomainFile=GrDomain.input; DomainDir={last_segment_path}/Run/););")

      # Change the Submit.sh run name
      replace_current_file(new_segment_path/"MakeSubmit.input",r"Jobname.*",f'Jobname = {folder_name}.{lev_dict["new_Lev_name"]}')
      replace_current_file(new_segment_path/"Submit.sh",r"#SBATCH -J.*",f'#SBATCH -J {folder_name}.{lev_dict["new_Lev_name"]}')

      change_AmrTolerances(new_segment_path, lev_dict['new_Lev_for_AMR_tolerance'], lev_dict['AH_factor'])
      change_GrDomain_and_AmrDriver_Level(new_segment_path, lev_dict['new_Lev_for_GrDomain_and_AmrDriver'])

    # Replace the text in files
    files_to_change_in_the_new_lev = lev_dict["files_to_change_in_the_new_lev"]
    for file_name in files_to_change_in_the_new_lev:
      file_path = new_segment_path/file_name
      if not file_path.exists():
        raise Exception(f"{file_path=} is supposed to be change in the new segment but it does not exist.")

      original_str_list = files_to_change_in_the_new_lev[file_name]["original_str"]
      replaced_str_list = files_to_change_in_the_new_lev[file_name]["replaced_str"]

      for original_str, replaced_str in zip(original_str_list,replaced_str_list):
        replace_current_file(file_path, original_str, replaced_str)


## Example usage

In [ ]:
# Ev is always made, the idea is to make things simple and consistent
data_dict={
  "6_set2_test":{
    "spec_home": Path("/home/hchaudha/spec"),
    "new_run_parent":Path("/groups/sxs/hchaudha/spec_runs/"),
    "old_Ev_path":Path("/groups/sxs/hchaudha/spec_runs/6_set2_test/Ev"),
    "link_or_copy_folders":"link",
    "Ev_is_present":True,
    "copy_ID":False,
    "copy_bin":False,
    "levs_to_copy":[
      {
        "old_Lev_name":"Lev3",
        "new_Lev_name":"Lev45",
        "this_lev_is_continuation": False,
        "new_Lev_for_AMR_tolerance":4.5,
        "new_Lev_for_GrDomain_and_AmrDriver":5,
        "AH_factor":1.0,
        "segments_to_copy":["AA"],
        "files_to_change_in_the_new_lev":{
          "Evolution.input":{
          "original_str": [
            r"FinalTime = \d*;",
            r"Tolerance =\s*([^;]*);"
          ],
          "replaced_str": [
            "FinalTime = 4000;",
            f"Tolerance = { 0.000216536/2000 * 4**(-4.5)};"
          ]
          },
          "Submit.sh":{
            "original_str": [
              "#SBATCH --constraint=skylake"
            ],
            "replaced_str": [
              "#SBATCH --constraint=skylake\n#SBATCH --reservation=sxs_standing"
            ]
          }
        },
      },
      {
        "old_Lev_name":"Lev3",
        "new_Lev_name":"Lev3",
        "this_lev_is_continuation": True,
        "AH_factor":1.0,
        "segments_to_copy":["AA"],
        "files_to_change_in_the_new_lev":{
          "Evolution.input":{
          "original_str": [
            r"FinalTime = \d*;",
            r"Tolerance =\s*([^;]*);"
          ],
          "replaced_str": [
            "FinalTime = 4000;",
            f"Tolerance = { 0.000216536/2000 * 4**(-3)};"
          ]
          },
          "Submit.sh":{
            "original_str": [
              "#SBATCH --constraint=skylake"
            ],
            "replaced_str": [
              "#SBATCH --constraint=skylake\n#SBATCH --reservation=sxs_standing"
            ]
          }
        },
      },
      {
        "old_Lev_name":"Lev3",
        "new_Lev_name":"Lev55",
        "this_lev_is_continuation": False,
        "new_Lev_for_AMR_tolerance":5.5,
        "new_Lev_for_GrDomain_and_AmrDriver":6,
        "AH_factor":1.0,
        "segments_to_copy":["AA"],
        "files_to_change_in_the_new_lev":{
          "Evolution.input":{
          "original_str": [
            r"FinalTime = \d*;",
            r"Tolerance =\s*([^;]*);"
          ],
          "replaced_str": [
            "FinalTime = 4000;",
            f"Tolerance = { 0.000216536/2000 * 4**(-5.5)};"
          ]
          },
          "Submit.sh":{
            "original_str": [
              "#SBATCH --constraint=skylake"
            ],
            "replaced_str": [
              "#SBATCH --constraint=skylake\n#SBATCH --reservation=sxs_standing"
            ]
          }
        },
      }
    ],
  }
}
# Ev is always made, the idea is to make things simple and consistent
data_dict={
  "L01_main_test":{
    "spec_home": Path("/home/hchaudha/spec"),
    "new_run_parent":Path("/groups/sxs/hchaudha/spec_runs/del"),
    "old_Ev_path":Path("/groups/sxs/hchaudha/spec_runs/Lev01_test/old_ode_tol/high_accuracy_L35_master/Ev"),
    "link_or_copy_folders":"link",
    "Ev_is_present":False,
    "copy_ID":False,
    "copy_bin":False,
    "levs_to_copy":[
      {
        "old_Lev_name":"Lev2",
        "new_Lev_name":"Lev45",
        "this_lev_is_continuation": False,
        "new_Lev_for_AMR_tolerance":4.5,
        "new_Lev_for_GrDomain_and_AmrDriver":5,
        "AH_factor":1.0,
        "segments_to_copy":["AA"],
        "files_to_change_in_the_new_lev":{
          "Evolution.input":{
          "original_str": [
            r"FinalTime = \d*;",
            r"Tolerance =\s*([^;]*);"
          ],
          "replaced_str": [
            "FinalTime = 4000;",
            f"Tolerance = { 0.000216536/2000 * 4**(-4.5)};"
          ]
          },
          "Submit.sh":{
            "original_str": [
              "#SBATCH --constraint=skylake"
            ],
            "replaced_str": [
              "#SBATCH --constraint=skylake\n#SBATCH --reservation=sxs_standing"
            ]
          }
        },
      },
      {
        "old_Lev_name":"Lev2",
        "new_Lev_name":"Lev3",
        "this_lev_is_continuation": False,
        "new_Lev_for_AMR_tolerance":3,
        "new_Lev_for_GrDomain_and_AmrDriver":3,
        "AH_factor":1.0,
        "segments_to_copy":["AA"],
        "files_to_change_in_the_new_lev":{
          "Evolution.input":{
          "original_str": [
            r"FinalTime = \d*;",
            r"Tolerance =\s*([^;]*);"
          ],
          "replaced_str": [
            "FinalTime = 4000;",
            f"Tolerance = { 0.000216536/2000 * 4**(-3)};"
          ]
          },
          "Submit.sh":{
            "original_str": [
              "#SBATCH --constraint=skylake"
            ],
            "replaced_str": [
              "#SBATCH --constraint=skylake\n#SBATCH --reservation=sxs_standing"
            ]
          }
        },
      },
      {
        "old_Lev_name":"Lev2",
        "new_Lev_name":"Lev55",
        "this_lev_is_continuation": False,
        "new_Lev_for_AMR_tolerance":5.5,
        "new_Lev_for_GrDomain_and_AmrDriver":6,
        "AH_factor":1.0,
        "segments_to_copy":["AA"],
        "files_to_change_in_the_new_lev":{
          "Evolution.input":{
          "original_str": [
            r"FinalTime = \d*;",
            r"Tolerance =\s*([^;]*);"
          ],
          "replaced_str": [
            "FinalTime = 4000;",
            f"Tolerance = { 0.000216536/2000 * 4**(-5.5)};"
          ]
          },
          "Submit.sh":{
            "original_str": [
              "#SBATCH --constraint=skylake"
            ],
            "replaced_str": [
              "#SBATCH --constraint=skylake\n#SBATCH --reservation=sxs_standing"
            ]
          }
        },
      },
      {
        "old_Lev_name":"Lev2",
        "new_Lev_name":"Lev2",
        "this_lev_is_continuation": True,
        "segments_to_copy":["AA"],
        "files_to_change_in_the_new_lev":{
          "Evolution.input":{
          "original_str": [
            r"FinalTime = \d*;",
            r"Tolerance =\s*([^;]*);"
          ],
          "replaced_str": [
            "FinalTime = 4000;",
            f"Tolerance = { 0.000216536/2000 * 4**(-5.5)};"
          ]
          },
          "Submit.sh":{
            "original_str": [
              "#SBATCH --constraint=skylake"
            ],
            "replaced_str": [
              "#SBATCH --constraint=skylake\n#SBATCH --reservation=sxs_standing"
            ]
          }
        },
      }
    ],
  }
}

In [ ]:
verify_data_dict(data_dict)

In [ ]:
# for key in data_dict:
#   create_new_folder(key,data_dict)

# Notebook part

### Use same ID

In [ ]:
# Ev is always made, the idea is to make things simple and consistent
data_dict = {
    "6_set2_test7": {
        "new_run_parent": Path("/resnick/groups/sxs/hchaudha/spec_runs/del"),
        "old_ID_path": Path("/resnick/groups/sxs/hchaudha/spec_runs/32_RM_set1_L3/ID"),
        "bin_dir_path": None,  # If None then ID.parent/Ev/bin will be used
        "input_files_source": None,  # If None then ID.parent()/Ev/*input files will be used
        "files_to_change": [  # All file paths should be relative to the new_Ev.parent() i.e. the base folder
            {
                "Ev/DoMultipleRuns.input": {
                    "original_str": [
                        r"FinalTime = \d*;",
                        r"TruncationErrorMax/\d*;",
                        r"MinLev = \d*;",
                        r"MaxLev = \d*;",
                        r"PBandJ = \d*;",
                    ],
                    "replaced_str": [
                        "FinalTime = 4000;",
                        "TruncationErrorMax/2000;",
                        "MinLev = 3;",
                        "MaxLev = 3;",
                        "PBandJ = 0;",
                    ],
                },
                "Ev/bin/Machines.pm": {
                    "original_str": [r"sub\s+HeaderSBATCH_CaltechHPC\s*\{.*?^\}"],
                    "replaced_str": [
                        r"""sub HeaderSBATCH_CaltechHPC {
  my ($ppn) = @_;
  my $header = HeaderSBATCH($ppn);
  if($ppn == 32) {
      $header .= "#SBATCH --constraint=skylake\n#SBATCH --cpus-per-task=1\n";
  } elsif($ppn == 56) {
      $header .= "#SBATCH --constraint=cascadelake\n#SBATCH --cpus-per-task=1\n";
  } elsif($ppn == 64) {
      $header .= "#SBATCH --constraint=icelake\n#SBATCH --cpus-per-task=1\n";
  } else {
      die "Unexpected ppn $ppn\n";
  }
  # Hello world
  return $header;
}"""
                    ],
                },
                "Ev/ConstraintDamping.input": {
                    "original_str": [
                        re.escape("""     EvaluateScalarFormula
     (Output=GhGamma0;
      Coords=GridToDistorted::MappedCoords;
      S = InvExpansionFactor;
      A = CDamping_distA_squared;
      B = CDamping_distB_squared;
      O = CDamping_distOrigin_squared;
      Formula =
      __CDamping_gamma0_AmpA__ *exp(-A/sqr(__CDamping_gamma0_WidthA__*S))
      +__CDamping_gamma0_AmpB__ *exp(-B/sqr(__CDamping_gamma0_WidthB__*S))
      +__CDamping_gamma0_AmpOrigin__
      *exp(-O/sqr(__CDamping_gamma0_WidthOrigin__*S))
      +__CDamping_gamma0_Asymptotic__;
      ),"""),
                        re.escape("""     EvaluateScalarFormula
     (Output=GhGamma2;
      Coords=GridToDistorted::MappedCoords;
      S = InvExpansionFactor;
      A = CDamping_distA_squared;
      B = CDamping_distB_squared;
      O = CDamping_distOrigin_squared;
      Formula =
      __CDamping_gamma2_AmpA__ *exp(-A/sqr(__CDamping_gamma2_WidthA__*S))
      +__CDamping_gamma2_AmpB__ *exp(-B/sqr(__CDamping_gamma2_WidthB__*S))
      +__CDamping_gamma2_AmpOrigin__
                              *exp(-O/sqr(__CDamping_gamma2_WidthOrigin__*S))
      +__CDamping_gamma2_Asymptotic__;
      )"""),
                    ],
                    "replaced_str": [
                        r"""     EvaluateScalarFormula
     (Output=GhGamma0;
      Coords=GridToDistorted::MappedCoords;
      S = InvExpansionFactor;
      A = CDamping_distA_squared;
      B = CDamping_distB_squared;
      O = CDamping_distOrigin_squared;
      Formula =
      __CDamping_gamma0_AmpA__ *exp(-A/sqr(__CDamping_gamma0_WidthA__*S))
      +__CDamping_gamma0_AmpB__ *exp(-B/sqr(__CDamping_gamma0_WidthB__*S))
      +__CDamping_gamma0_AmpOrigin__
      *exp(-O/sqr(__CDamping_gamma0_WidthOrigin__*S))
      +__CDamping_gamma0_Asymptotic__;
      ),""",
                        r"""     EvaluateScalarFormula
     (Output=GhGamma2;
      Coords=GridToDistorted::MappedCoords;
      S = InvExpansionFactor;
      A = CDamping_distA_squared;
      B = CDamping_distB_squared;
      O = CDamping_distOrigin_squared;
      Formula =
      __CDamping_gamma2_AmpA__ *exp(-A/sqr(__CDamping_gamma2_WidthA__*S))
      +__CDamping_gamma2_AmpB__ *exp(-B/sqr(__CDamping_gamma2_WidthB__*S))
      +__CDamping_gamma2_AmpOrigin__
                              *exp(-O/sqr(__CDamping_gamma2_WidthOrigin__*S))
      +__CDamping_gamma2_Asymptotic__;
      )""",
                    ],
                },
            },
        ],
    }
}


In [ ]:
# Ev is always made, the idea is to make things simple and consistent
data_dict = {
    "6_set2_test7": {
        "new_run_parent": Path("/resnick/groups/sxs/hchaudha/spec_runs/del"),
        "old_ID_path": Path("/resnick/groups/sxs/hchaudha/spec_runs/32_RM_set1_L3/ID"),
        "bin_dir_path": None,  # If None then ID.parent/Ev/bin will be used
        "input_files_source": None,  # If None then ID.parent()/Ev/*input files will be used
        "files_to_change": [  # All file paths should be relative to the new_Ev.parent() i.e. the base folder
            {
                "Ev/DoMultipleRuns.input": {
                    "original_str": [
                        r"FinalTime = \d*;",
                        r"TruncationErrorMax/\d*;",
                        r"MinLev = \d*;",
                        r"MaxLev = \d*;",
                        r"PBandJ = \d*;",
                    ],
                    "replaced_str": [
                        "FinalTime = 4000;",
                        "TruncationErrorMax/2000;",
                        "MinLev = 3;",
                        "MaxLev = 3;",
                        "PBandJ = 0;",
                    ],
                },
                "Ev/bin/Machines.pm": {
                    "original_str": [r"sub\s+HeaderSBATCH_CaltechHPC\s*\{.*?^\}"],
                    "replaced_str": [
                        r"""sub HeaderSBATCH_CaltechHPC {
  my ($ppn) = @_;
  my $header = HeaderSBATCH($ppn);
  if($ppn == 32) {
      $header .= "#SBATCH --reservation=sxs_standing\n#SBATCH --cpus-per-task=1\n";
  } elsif($ppn == 56) {
      $header .= "#SBATCH --reservation=sxs_standing\n#SBATCH --cpus-per-task=1\n";
  } elsif($ppn == 64) {
      $header .= "#SBATCH --reservation=sxs_standing\n#SBATCH --cpus-per-task=1\n";
  } else {
      die "Unexpected ppn $ppn\n";
  }
  # Hello world
  return $header;
}"""
                    ],
                },
                "Ev/ConstraintDamping.input": {
                    "original_str": [
                        re.escape("""     EvaluateScalarFormula
     (Output=GhGamma0;
      Coords=GridToDistorted::MappedCoords;
      S = InvExpansionFactor;
      A = CDamping_distA_squared;
      B = CDamping_distB_squared;
      O = CDamping_distOrigin_squared;
      Formula =
      __CDamping_gamma0_AmpA__ *exp(-A/sqr(__CDamping_gamma0_WidthA__*S))
      +__CDamping_gamma0_AmpB__ *exp(-B/sqr(__CDamping_gamma0_WidthB__*S))
      +__CDamping_gamma0_AmpOrigin__
      *exp(-O/sqr(__CDamping_gamma0_WidthOrigin__*S))
      +__CDamping_gamma0_Asymptotic__;
      ),"""),
                        re.escape("""     EvaluateScalarFormula
     (Output=GhGamma2;
      Coords=GridToDistorted::MappedCoords;
      S = InvExpansionFactor;
      A = CDamping_distA_squared;
      B = CDamping_distB_squared;
      O = CDamping_distOrigin_squared;
      Formula =
      __CDamping_gamma2_AmpA__ *exp(-A/sqr(__CDamping_gamma2_WidthA__*S))
      +__CDamping_gamma2_AmpB__ *exp(-B/sqr(__CDamping_gamma2_WidthB__*S))
      +__CDamping_gamma2_AmpOrigin__
                              *exp(-O/sqr(__CDamping_gamma2_WidthOrigin__*S))
      +__CDamping_gamma2_Asymptotic__;
      )"""),
                    ],
                    "replaced_str": [
                        r"""     EvaluateScalarFormula
     (Output=GhGamma0;
      Coords=GridToDistorted::MappedCoords;
      S = InvExpansionFactor;
      A = CDamping_distA_squared;
      B = CDamping_distB_squared;
      O = CDamping_distOrigin_squared;
      Formula =
      __CDamping_gamma0_AmpA__ *exp(-A/sqr(__CDamping_gamma0_WidthA__*S))
      +__CDamping_gamma0_AmpB__ *exp(-B/sqr(__CDamping_gamma0_WidthB__*S))
      +__CDamping_gamma0_AmpOrigin__
      *exp(-O/sqr(__CDamping_gamma0_WidthOrigin__*S))
      +__CDamping_gamma0_Asymptotic__;
      ),""",
                        r"""     EvaluateScalarFormula
     (Output=GhGamma2;
      Coords=GridToDistorted::MappedCoords;
      S = InvExpansionFactor;
      A = CDamping_distA_squared;
      B = CDamping_distB_squared;
      O = CDamping_distOrigin_squared;
      Formula =
      __CDamping_gamma2_AmpA__ *exp(-A/sqr(__CDamping_gamma2_WidthA__*S))
      +__CDamping_gamma2_AmpB__ *exp(-B/sqr(__CDamping_gamma2_WidthB__*S))
      +__CDamping_gamma2_AmpOrigin__
                              *exp(-O/sqr(__CDamping_gamma2_WidthOrigin__*S))
      +__CDamping_gamma2_Asymptotic__;
      )""",
                    ],
                },
            },
        ],
    }
}


In [ ]:
for folder_name in data_dict.keys():
    assert data_dict[folder_name]["new_run_parent"].exists(), (
        f"{data_dict[folder_name]['new_run_parent']} does not exist!"
    )

    base_folder_path = data_dict[folder_name]["new_run_parent"] / folder_name
    base_folder_path.mkdir(exist_ok=False)

    # Copy the ID folder
    new_ID_path = base_folder_path / "ID"
    old_ID_path = data_dict[folder_name]["old_ID_path"]
    if old_ID_path.exists():
        shutil.copytree(old_ID_path, new_ID_path, symlinks=True)
    else:
        raise Exception(f"{old_ID_path} does not exist!")

    # Add write permission to the ID folder
    add_write_permission(new_ID_path)

    new_bin_path = new_ID_path / "bin"
    old_bin_path = data_dict[folder_name]["bin_dir_path"]
    if old_bin_path is None:
        old_bin_path = old_ID_path.parent / "Ev/bin"
    if not old_bin_path.exists():
        raise Exception(f"{old_bin_path} does not exist!")

    # Copy the bin folder
    shutil.copytree(old_bin_path, new_bin_path)

    # Create the new Ev folder
    new_Ev_path = base_folder_path / "Ev"
    new_Ev_path.mkdir(exist_ok=False)

    # Copy the bin dir into the new Ev folder
    new_Ev_bin_path = new_Ev_path / "bin"
    shutil.copytree(old_bin_path, new_Ev_bin_path)

    # Copy the input files
    input_files_source = data_dict[folder_name]["input_files_source"]
    if input_files_source is None:
        input_files_source = old_ID_path.parent / "Ev"
    if not input_files_source.exists():
        raise Exception(f"{input_files_source=}folder does not exist!")

    # Copy the input files to the new Ev folder
    for file in input_files_source.glob("*.input"):
        shutil.copy2(file, new_Ev_path)

    # Make DoMultipleRuns.input executable
    DMR = new_Ev_path / "DoMultipleRuns.input"
    DMRR = new_Ev_path / "DoMultipleRunsRingdown.input"

    for i in [DMR, DMRR]:
        current_permissions = i.stat().st_mode
        # Add execute permission for the user and group
        new_permissions = current_permissions | stat.S_IXUSR | stat.S_IXGRP
        i.chmod(new_permissions)

    # Symlink Ev/bin/EvolveHyperbolicSystem to SpEC
    new_SpEC_symlink = new_Ev_path / "SpEC"
    new_SpEC_symlink.symlink_to("bin/EvolveHyperbolicSystem")

    new_StartJob_sh_path = new_Ev_path / "StartJob.sh"
    with open(new_StartJob_sh_path, "w") as f:
        f.write("""#!/bin/bash
. bin/this_machine.env || echo 'Not using env file.'
# Only applicable for bbh2 runs:
# For PBandJ (Perform Branching after Junk) only EccRedLev should be started
# with StartJob.sh. ProhibitStartJobReruns.txt gets generated at Eccentricity
# Reduction restart, and safeguards against starting Levs != EccRedLev from t=0
# for PBandJ.
if test -f ProhibitStartJobReruns.txt; then
    echo
    echo "ERROR: ProhibitStartJobReruns.txt exists. See below:"
    cat ProhibitStartJobReruns.txt
    echo
    exit 1
fi

bin/DoMultipleRuns -L -c 'sbatch ./Submit.sh'""")
    # Make StartJob.sh executable
    current_permissions = new_StartJob_sh_path.stat().st_mode
    # Add execute permission for the user and group
    new_permissions = current_permissions | stat.S_IXUSR | stat.S_IXGRP
    new_StartJob_sh_path.chmod(new_permissions)

    # Add write permission to the new Ev folder and the new bin folder
    add_write_permission(base_folder_path)

    print(
        f"\n\nCreated new run folder at {base_folder_path}. Making changes to the input files now....\n\n"
    )

    for file in data_dict[folder_name]["files_to_change"]:
        for file_name, changes in file.items():
            file_path = base_folder_path / file_name
            if not file_path.exists():
                raise Exception(f"{file_path} does not exist!")
            original_str = changes["original_str"]
            replaced_str = changes["replaced_str"]
            for or_str, re_str in zip(original_str, replaced_str):
                replace_current_file(
                    file_path, or_str, re_str, flags=re.DOTALL | re.MULTILINE
                )
